In [1]:
import models

c:\Users\aruav\Desktop\Deepen\CTG\cambio\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
models.predict_ipca()
models.predict_selic()
models.predict_cambio()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False

,prediction,superior,inferior,std
date,,,,
2022-12,5.267450,6.649463,3.885437,0.520584
2023-01,4.926185,6.218661,3.633709,0.520584
2023-02,5.026240,6.344967,3.707513,0.520584
2023-03,4.987378,6.295908,3.678847,0.520584
2023-04,4.840602,6.110624,3.570581,0.520584
2023-05,5.402469,6.819907,3.985031,0.520584
2023-06,5.274653,6.658555,3.890750,0.520584
2023-07,5.809473,7.333696,4.285250,0.520584
2023-08,5.646135,7.127503,4.164767,0.520584
